# Continuous Bag of Words (CBOW) Model

In this exam, we will create a CBOW model using a sample Arabic traffic corpus. The corpus consists of sentences describing various traffic scenarios. The goal of the CBOW model is to predict a target word based on its surrounding context words.

# Importing Required Libraries

In this step, we import the necessary libraries to build and train the Continuous Bag of Words (CBOW) model.

- **TensorFlow and Keras**: Used to build the neural network model, including the layers like `Embedding`, `Dense`, and `Lambda`.
- **Tokenizer**: A utility from Keras for tokenizing and processing text data.
- **NumPy**: Used for handling numerical operations, particularly for processing arrays and data manipulation.

These libraries will provide the essential tools for text preprocessing and model development in the upcoming steps.

Add more if needed!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Preparing the Corpus

In this step, we initialize the corpus that will be used for training the Continuous Bag of Words (CBOW) model. The corpus consists of Arabic sentences, each of which describes different traffic scenarios.

- **Corpus**: A collection of traffic-related sentences in Arabic.

This step sets up the text data that we will use in the upcoming stages of tokenization and model training.

In [ ]:
corpus = [
    "الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول",
    "كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن",
    "أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق",
    "السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة",
    "هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير",
    "ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل",
    "الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير",
    "الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم",
    "حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك",
    "كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذي استمر طوال اليوم",
    "الطريق إلى المطار مزدحم اليوم بسبب الحوادث المتكررة والتأخيرات الكبيرة في حركة المرور على الطريق السريع",
    "الشارع مزدحم بالسيارات والحافلات الكبيرة مما يجعل التنقل بطيئًا جدًا ويزيد من وقت الوصول إلى العمل",
    "ازدحام السيارات في المدينة أصبح مشكلة كبيرة خاصة خلال ساعات الذروة حيث يصعب التحرك بسرعة",
    "تفاقم الازدحام في الطرق الجانبية بسبب إغلاق الطريق الرئيسي المؤدي إلى وسط المدينة لصيانة الجسر",
    "ازدحام مروري خانق يواجه السكان يوميًا خلال تنقلهم من وإلى العمل على الطرق السريعة المؤدية إلى المدينة",
    "التأخيرات المرورية اليوم ناجمة عن سوء الأحوال الجوية والضباب الذي يعيق الرؤية ويبطئ حركة السيارات",
    "حوادث السير المتكررة على الطريق الزراعي تؤدي إلى ازدحام مروري شديد وتأخير كبير في وصول السيارات",
    "كانت حركة السير اليوم غير منتظمة بسبب تنظيم حدث رياضي كبير أدى إلى إغلاق بعض الشوارع الرئيسية",
    "الأعمال الإنشائية في الشارع الرئيسي تسببت في اختناق مروري كامل وتباطؤ في حركة السيارات خلال النهار",
    "تراكم السيارات عند تقاطع الطرق الرئيسية أدى إلى ازدحام شديد وزيادة كبيرة في مدة الانتظار للوصول إلى الجهة المطلوبة"
]

# Defining Vocabulary and Model Parameters

In this step, we define key parameters that will be used to configure the CBOW model.

- **Vocabulary size**: We calculate the size of the vocabulary based on the number of unique words in the corpus. The `vocab_size` represents the total number of unique tokens (words) in the dataset plus one for padding.
  
- **Embedding size**: The `embedding_size` defines the dimensionality of the word embeddings. In this case, we set the embedding size to 10, meaning each word will be represented as a 10-dimensional vector in the embedding layer.

- **Window size**: The `window_size` defines how many words to the left and right of the target word are considered as context. Here, a window size of 2 means that two words before and two words after the target word will be used as context.

These parameters will play an essential role in shaping the CBOW model architecture.


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size = 2

# Preparing Context-Target Pairs for CBOW

In this step, we generate the context-target pairs from the tokenized sequences to train the CBOW model.

- **Context words**: For each word in a sequence, the surrounding words (within the window size) are considered as context. The context consists of the words immediately before and after the target word.
  
- **Target word**: The word in the middle of the context window is treated as the target word that the model will learn to predict.

We iterate through each sequence, collecting the context words and corresponding target words:
- For each word in a sequence, we gather the surrounding words based on the defined window size.
- The middle word is the target, and the surrounding words form the context.

Finally:
- **`X`**: An array of context words.
- **`y`**: The target words are one-hot encoded, which means they are converted into a categorical format where each word is represented as a vector of length equal to the vocabulary size.

These context-target pairs will be used to train the CBOW model to predict a target word based on its context.


In [ ]:
sequences = tokenizer.texts_to_sequences(corpus)

In [ ]:
contexts = []
targets = []
for sequence in sequences:
    for i in range(window_size, len(sequence) - window_size):
        context = sequence[i-window_size:i] + sequence[i+1:i+window_size+1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)
X = np.array(contexts)
y = tf.keras.utils.to_categorical(targets, num_classes=vocab_size)

# Building and Training the CBOW Model

In this step, we build and train the Continuous Bag of Words (CBOW) model using the context-target pairs created earlier.

1. **Model architecture**:
   - **Embedding layer**: This layer transforms the input context words into dense vector representations (embeddings) of size defined by `embedding_size`. The `input_dim` is set to the vocabulary size, and the `input_length` is twice the window size (since context consists of words from both sides of the target).
   
   - **Lambda layer**: This layer computes the mean of the context word embeddings. It averages the embeddings of the context words to generate a single representation that will be used to predict the target word.
   
   - **Dense layer**: This fully connected layer outputs a probability distribution over the vocabulary, using the softmax activation function. It predicts the most likely target word based on the context word embeddings.

2. **Compilation**:
   The model is compiled using the Adam optimizer and categorical cross-entropy as the loss function, which is suitable for multi-class classification tasks. Accuracy is used as a metric to evaluate the model's performance during training.

3. **Training the model**:
   The model is trained on the context-target pairs for 500 epochs. During each epoch, the model learns to predict the target word based on the context, refining its weights to improve accuracy.

4. **Saving the model weights**:
   After training, the model weights are saved to a file (`cbow_model.weights.h5`) for future use. This allows us to load the trained model later without retraining.

By the end of this step, the CBOW model will have learned to predict target words based on their surrounding context from the given corpus.

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(units=vocab_size, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_3 (Lambda)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=500, verbose=1)

In [ ]:
model.save_weights('cbow_model.weights.h5')

# Predicting a Word Using the Trained CBOW Model

In this step, we define a function to predict a word based on a given context using the trained CBOW model.

1. **Function: `predict_word`**:
   - **Input**: The function takes a list of context words as input. The number of context words should match the expected size (2 times the window size).
   - **Context sequence conversion**: The input context words are tokenized into a sequence of integers using the same tokenizer that was used during training.
   - **Input validation**: The function checks whether the length of the context sequence matches the expected size (2 times the window size). If not, it prints an error message.
   - **Prediction**: The tokenized context is fed into the trained CBOW model, which predicts the probability distribution over the vocabulary.
   - **Retrieve predicted word**: The predicted word is the one with the highest probability. The function retrieves the word corresponding to the predicted index from the tokenizer's word index.

2. **Example**:
   - We provide a sample context: `['الحادث', 'بسبب', 'مزدحم', 'الطريق']`.
   - The function predicts the word that fits best in this context, based on the model's learned weights.
   - The predicted word is printed along with the input context.

This function allows us to test the CBOW model by predicting words based on their surrounding context from the corpus.

In [ ]:
def predict_word(context):
    context_sequence = tokenizer.texts_to_sequences([context])
    context_sequence = np.array(context_sequence)
    prediction = model.predict(context_sequence)
    predicted_index = np.argmax(prediction)
    predicted_word = tokenizer.index_word[predicted_index]
    print(f"Context: {context}")
    print(f"Predicted word: {predicted_word}")
    return predicted_word




In [ ]:
predict_word(['الطريق'])
predict_word(['مزدحم'])
predict_word(['بسبب'])
predict_word([ 'الحادث'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Context: ['الطريق']
Predicted word: على
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Context: ['مزدحم']
Predicted word: اليوم
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Context: ['بسبب']
Predicted word: غير
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Context: ['الحادث']
Predicted word: بسبب


'بسبب'